# Residual Diagnostic Example

<div class="alert alert-info">

This module is newly developed and still experimental. Further refinement should be found in future version.

</div>

## Library an Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import orbit
from orbit.models import DLT
from orbit.diagnostics.plot import residual_diagnostic_plot
from orbit.utils.dataset import load_iclaims
import pandas as pd
import seaborn as sns

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
print(orbit.__version__)

1.1.3dev


In [3]:
# load log-transformed data
df = load_iclaims()
response_col = 'claims'
date_col = 'week'

In [4]:
df.head()

,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


## Model Fitting

In [5]:
dlt = DLT(
    response_col=response_col,
    date_col=date_col,
    estimator='stan-map',
    seed=8888,
    seasonality=52,
    regressor_col=['trend.unemploy', 'trend.filling'],
    regressor_beta_prior=[0.1, 0.3],
    regressor_sigma_prior=[0.5, 2.0],
)

dlt.fit(df)
predicted_df = dlt.predict(df, decompose=False)

INFO:orbit:Optimizing (PyStan) with algorithm: LBFGS.


## Residual Diagnostic

In [15]:
# compute residuals
predicted_df['resid'] = df['claims'] - predicted_df['prediction'] 

residual_diagnostic_plot(
    df=predicted_df,
    dist='t-dist',
    sparams=dlt.get_point_posteriors()['map']['nu'][0],
    residual_col='resid',
)

INFO:root:Cannot find path:/local/home/edng/work/ts-research/orbit/orbit/stylelib/plot_style.mplstyle. Use default plot style.


TypeError: residual_diagnostic_plot() got an unexpected keyword argument 'dist'